In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
races_df = spark.read.table("f1_processed.races") \
    .withColumnRenamed("name", "race_name") \
    .withColumnRenamed("race_timestamp", "race_date")

In [0]:
circuits_df = spark.read.table("f1_processed.circuits") \
    .withColumnRenamed("location", "circuit_location") \
    .withColumnRenamed("name", "circuit_name")

In [0]:
drivers_df = spark.read.table("f1_processed.drivers") \
    .withColumnRenamed("name", "driver_name") \
    .withColumnRenamed("number", "driver_number") \
    .withColumnRenamed("nationality", "driver_nationality")

In [0]:
results_df = spark.read.table("f1_processed.results") \
    .filter(f"file_date = '{v_file_date}'")\
    .withColumnRenamed("time", "race_time") \
            .withColumnRenamed("race_Id", "results_race_id") \
                .withColumnRenamed("file_date", "results_file_date")

In [0]:
constructors_df = spark.read.table("f1_processed.constructors") \
    .withColumnRenamed("name", "team")

In [0]:
races_circuits_df = races_df.join(circuits_df, races_df.circuit_id == circuits_df.circuit_id, "inner").select(races_df.race_id, races_df.race_year, races_df.race_name, races_df.race_date, circuits_df.circuit_location)

In [0]:
races_circuits_results_df = races_circuits_df.join(results_df, races_circuits_df.race_id == results_df.results_race_id, "inner")

In [0]:
races_circuits_drivers_results_df = races_circuits_results_df.join(drivers_df, races_circuits_results_df.driver_id == drivers_df.driver_id, "inner")

In [0]:
races_circuits_drivers_results_constructors_df = races_circuits_drivers_results_df.join(constructors_df, races_circuits_drivers_results_df.constructor_id == constructors_df.constructor_id, "inner")

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
final_df = races_circuits_drivers_results_constructors_df.select("race_id","race_year", "race_name", "race_date", "circuit_location", "driver_name", "driver_number", "driver_nationality", "team", "grid", "fastest_lap", "race_time", "points","position","results_file_date") \
    .withColumn("created_date", current_timestamp()) \
    .withColumnRenamed("results_file_date", "file_date")

In [0]:
merge_condition = "tgt.driver_name = src.driver_name AND tgt.race_id = src.race_id"
merge_delta_data(final_df,'f1_presentation', 'race_results', merge_condition,'race_id')